# Sincronización de Puente

Ernesto Rivera Gálvez

En una carretera de doble sentido llegamos a un punto donde hay un puente, en el puente solo cabe un carril y a lo  mucho puede tener 3 autos, de otro modo el puente cae.

In [1]:
import multiprocessing as mp
import time
import random

Podemos ver el puente con tres casillas, pues máximo puede haber tres carros o se cae. Además, cada casilla solo puede estar ocupada por un solo auto, esto hace cada casilla una sección crítica. Finalmente, necesitamos saber en qué dirección están pasando los carros, pues no pueden pasar en ambas direcciones y si el puente está libre, para saber si es posible cambiar la dirección

Definiendo esto en términos de sincronización, podemos ver cada casilla como un semáforo binario, el puente como un semáforo no binario de valor 3 y finalmente podemos ver la dirección como una bandera, que indique en qué dirección se está circulando y otra bandera para saber si está libre el puente

Haremos una clase carro, únicamente para tener los valores de la dirección a la que va y un id

Implementando lo anterior tenemos

In [37]:
class Carro():
  def __init__(self, id):
    self.id = id
    self.dir = random.randint(0,1)
    self.no_pasado = True

In [30]:
# Ejemplo de un carrito
carro = Carro(0)
print(carro.id, carro.dir)

0 0


In [35]:
def cruza_puente(dirección, ocupado, casillas, puente, carrito):
  print(f"Carrito {carrito.id} llegando en dirección \
    {'Norte' if carrito.dir else 'Sur'}")

  while carrito.no_pasado: # Mientras no haya pasado el puente
    if not ocupado.value: # Si el puente está vacío
      ocupado.value = 1
      dirección.value = carrito.dir # El puente tendrá la dirección del carro

    # Si el puente y el carro tienen la misma dirección
    if carrito.dir == dirección.value:
      print(f"{carrito.id} por entrar al puente")

      with puente: # entramos al puente 
        for i, casilla in enumerate(casillas):
          print(f"{carrito.id} tratando de avanzar a casilla {i}")

          with casilla: # Entramos a la casilla
            print(f"\t{carrito.id} entró a {i}")
            time.sleep(random.randint(0, 3)) # Tiempo en pasar el puente
          print(f"\t{carrito.id} pasó la casilla {i}")
          
        carrito.no_pasado = False # Después de pasar todas las casillas

    if puente.get_value() == 3: # Si el puente está vacío
      ocupado.value = 0 # Lo indicamos en su variable

    time.sleep(random.randint(0, 3))

In [36]:
if __name__ == '__main__':
    num_carritos = 4
    carritos = [Carro(i) for i in range(num_carritos)]
    casillas = [mp.Semaphore() for _ in range(3)]
    puente = mp.Semaphore(len(casillas))
    ocupado = mp.Value('i', 0)
    direccion = mp.Value('i', 0)

    procesos = []
    for i in range(num_carritos):
        procesos.append(mp.Process(target=cruza_puente,
                                   args=(direccion,
                                        ocupado,
                                        casillas,
                                        puente,
                                        carritos[i],)))
    for proceso in procesos:
        proceso.start()

    for proceso in procesos:
        proceso.join()

Carrito 0 llegando en dirección     Norte
0 por entrar al puenteCarrito 1 llegando en dirección     Norte

0 tratando de avanzar a casilla 01 por entrar al puente
Carrito 2 llegando en dirección     Sur	0 entró a 0


Carrito 3 llegando en dirección     Sur
1 tratando de avanzar a casilla 0
	1 entró a 0	0 pasó la casilla 0

0 tratando de avanzar a casilla 1
	0 entró a 1
	0 pasó la casilla 1
0 tratando de avanzar a casilla 2
	0 entró a 2
	0 pasó la casilla 2
	1 pasó la casilla 0
1 tratando de avanzar a casilla 1
	1 entró a 1
	1 pasó la casilla 1
1 tratando de avanzar a casilla 2
	1 entró a 2
	1 pasó la casilla 2
3 por entrar al puente
3 tratando de avanzar a casilla 0
	3 entró a 0
	3 pasó la casilla 0
3 tratando de avanzar a casilla 1
	3 entró a 1
	3 pasó la casilla 1
3 tratando de avanzar a casilla 2
	3 entró a 2
2 por entrar al puente
2 tratando de avanzar a casilla 0
	2 entró a 0
	2 pasó la casilla 0
2 tratando de avanzar a casilla 1
	2 entró a 1
	2 pasó la casilla 1
2 tratando de ava